# Nemo 3D tutorial


One of the features of Parcels is that it can directly and natively work with `Field` data discretised on C-grids. These C grids are very popular in OGCMs, so velocity fields outputted by OGCMs are often provided on such grids, except if they have been firstly re-interpolated on a A grid.

More information about C-grid interpolation can be found in [Delandmeter et al., 2019](https://www.geosci-model-dev-discuss.net/gmd-2018-339/).
An example of such a discretisation is the NEMO model, which is one of the models supported in Parcels. A tutorial teaching how to [interpolate 2D data on a NEMO grid](https://docs.oceanparcels.org/en/latest/examples/tutorial_nemo_curvilinear.html) is available within Parcels.

```{note}
_How to know if your data is discretised on a C grid?_ The best way is to read the documentation that comes with the data. Alternatively, an easy check is to assess the coordinates of the U, V and W fields: for an A grid, U, V and W are distributed on the same nodes, such that the coordinates are the same. For a C grid, there is a shift of half a cell between the different variables.
```

Here, we focus on 3D fields. Basically, it is a straightforward extension of the 2D example, but it is very easy to make a mistake in the setup of the vertical discretisation that would affect the interpolation scheme.

For the C-grid interpolation in Parcels to work properly, it is important that `U`, `V` and `W` are on the same grid. All other tracers (e.g. `temperature`, `salinity`) should also be on this same grid. So even though these tracers are computed by NEMO on the T-points, Parcels expects them on the f-points (`glamf`, `gphif` and `depthw`). Parcels then under the hood makes sure the interpolation of these tracers is done correctly.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

Parcels v4 comes with a `from_nemo` method that automatically sets up the correct `Grid` for both 2D and 3D NEMO data. However, you may need to do some data-wrangling on your NEMO data files, such as below:

In [ ]:
data_folder = parcels.download_example_dataset("NemoNorthSeaORCA025-N006_data")

# Open field datasets (with minimal loading to speed up)
ds_fields = xr.open_mfdataset(
    data_folder.glob("ORCA*.nc"),
    data_vars="minimal",
    coords="minimal",
    compat="override",
)

# Open coordinates dataset (with no time decoding because xarray otherwise complains)
ds_coords = xr.open_dataset(data_folder / "coordinates.nc", decode_times=False)

# Remove time dimension from coordinates (otherwise xarray complains)
ds_coords = ds_coords.isel(time=0, drop=True)

# Combine field and coordinate datasets
ds = xr.merge([ds_fields, ds_coords[["glamf", "gphif"]]])

fieldset = parcels.FieldSet.from_nemo(ds)

The code below is an example of how to then create a 3D simulation with particles, starting on a meridional line through the North Sea from the mouth of the river Rhine at 100m depth, and advecting them through the North Sea using the `AdvectionRK2_3D`

In [ ]:
npart = 10
pset = parcels.ParticleSet(
    fieldset=fieldset,
    lon=np.linspace(1.9, 3.4, npart),
    lat=np.linspace(65, 51.6, npart),
    z=100 * np.ones(npart),
)

pfile = parcels.ParticleFile(
    store="output_nemo3D.zarr", outputdt=np.timedelta64(1, "D")
)

pset.execute(
    parcels.kernels.AdvectionRK2_3D,
    endtime=fieldset.time_interval.right,
    dt=np.timedelta64(6, "h"),
    output_file=pfile,
)

We can then plot the trajectories on top of the surface U field

In [ ]:
field = fieldset.U.data[0, 0, :, :]
field = field.where(field != 0, np.nan)  # Mask land values for better plotting
plt.pcolormesh(fieldset.U.grid.lon, fieldset.U.grid.lat, field, cmap="RdBu")

ds_out = xr.open_zarr("output_nemo3D.zarr")
plt.scatter(ds_out.lon.T, ds_out.lat.T, c=ds_out.z.T, marker=".")
plt.show()